# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
# your code here
import zipfile

zip_file_path = '/Users/amandine/Desktop/Ironhack/02_Week/lab-dataframe-calculations/your-code/Orders.zip'

# Define the path where you want to extract the contents
extracted_folder_path = '/Users/amandine/Desktop/Ironhack/02_Week/lab-dataframe-calculations/your-code'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

csv_file_path = f'{extracted_folder_path}/Orders.csv'

orders = pd.read_csv(csv_file_path)

# Print the head of the 'orders' DataFrame
orders.head(3)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
# your code here
customer_spending = orders.groupby('CustomerID')['amount_spent'].sum()

# Print the first few rows to see the aggregated data
customer_spending.head()

CustomerID
12346    77183.60
12347     4310.00
12348     1797.24
12349     1757.55
12350      334.40
Name: amount_spent, dtype: float64

In [20]:
orders_customer_spending_added = pd.merge(left = orders,
         right = customer_spending,
         left_on = 'CustomerID',
         right_on = 'CustomerID',
         how = 'inner')
orders_customer_spending_added = orders_customer_spending_added.rename(columns={'amount_spent_x': 'AmountSpent_perArticle', 'amount_spent_y': 'CustomerTotalSpent'})

orders_customer_spending_added.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,AmountSpent_perArticle,CustomerTotalSpent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,5391.21
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,5391.21
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21


In [23]:
selected_columns = ['InvoiceNo', 'CustomerID', 'Country', 'CustomerTotalSpent']

# Create a new DataFrame with only the selected columns
orders_selected = orders_customer_spending_added[selected_columns]
orders_selected.head()

,InvoiceNo,CustomerID,Country,CustomerTotalSpent
0,536365,17850,United Kingdom,5391.21
1,536365,17850,United Kingdom,5391.21
2,536365,17850,United Kingdom,5391.21
3,536365,17850,United Kingdom,5391.21
4,536365,17850,United Kingdom,5391.21


In [26]:
customer_grouped = orders_selected.groupby(['CustomerID', 'Country'])['CustomerTotalSpent'].sum().reset_index()
customer_grouped.head(5)

,CustomerID,Country,CustomerTotalSpent
0,12346,United Kingdom,77183.60
1,12347,Iceland,784420.00
2,12348,Finland,55714.44
3,12349,Italy,128301.15
4,12350,Norway,5684.80


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [27]:
# your code here
percentiles = customer_grouped['CustomerTotalSpent']

# Calculate the 95th percentile
percentile_95 = np.percentile(percentiles, 95)

# Calculate the 75th percentile
percentile_75 = np.percentile(percentiles, 75)

print(f"95th percentile: {percentile_95}")
print(f"75th percentile: {percentile_75}")

95th percentile: 1543490.4719999996
75th percentile: 151660.065


In [29]:
top_customers = pd.DataFrame(customer_grouped, columns=['CustomerID', 'Country', 'CustomerTotalSpent'])

def best_customers(row, percentile_75, percentile_95):
    if row['CustomerTotalSpent'] > percentile_95:
        return 'VIP'
    elif percentile_75 <= row['CustomerTotalSpent'] <= percentile_95:
        return 'Preferred customer'
    else:
        return 'Regular customer'
    
top_customers['TopCustomer'] = top_customers.apply(best_customers, args=(percentile_75, percentile_95), axis=1)

top_customers.head(25)

,CustomerID,Country,CustomerTotalSpent,TopCustomer
0,12346,United Kingdom,77183.60,Regular customer
1,12347,Iceland,784420.00,Preferred customer
2,12348,Finland,55714.44,Regular customer
3,12349,Italy,128301.15,Regular customer
4,12350,Norway,5684.80,Regular customer
5,12352,Norway,213013.40,Preferred customer
6,12353,Bahrain,356.00,Regular customer
7,12354,Spain,62605.20,Regular customer
8,12355,Bahrain,5972.20,Regular customer
9,12356,Portugal,165874.37,Preferred customer


In [38]:
#If you want to reset the index and get the result as a DataFrame, you can use .reset_index()

value_counts = top_customers.groupby(['Country', 'TopCustomer'])['CustomerID'].count().reset_index()
max_vip_country = value_counts[value_counts['TopCustomer'] == 'VIP'].sort_values(by='CustomerID', ascending=False).iloc[0]

max_vip_country.head(10)

Country        United Kingdom
TopCustomer               VIP
CustomerID                183
Name: 71, dtype: object

In [40]:
value_counts = top_customers.groupby(['Country', 'TopCustomer'])['CustomerID'].count().reset_index()
max_vip_country = value_counts[value_counts['TopCustomer'] == 'Preferred customer'].sort_values(by='CustomerID', ascending=False).iloc[0]

max_vip_country.head(10)

Country            United Kingdom
TopCustomer    Preferred customer
CustomerID                    768
Name: 69, dtype: object

In [30]:
country_counts = top_customers[top_customers['TopCustomer'].isin(['VIP'])].groupby('Country').size().reset_index(name='CustomerCount')

# Find the country with the most VIP and Preferred Customers combined
most_customers_country = country_counts.loc[country_counts['CustomerCount'].idxmax()]

print("Country with the most VIP Customers is:", most_customers_country['Country'])

Country with the most VIP Customers is: United Kingdom


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [39]:
# your code here
country_counts = top_customers[top_customers['TopCustomer'].isin(['VIP', 'Preferred customer'])].groupby('Country').size().reset_index(name='CustomerCount')

# Find the country with the most VIP and Preferred Customers combined
most_customers_country = country_counts.loc[country_counts['CustomerCount'].idxmax()]

print("Country with the most VIP and Preferred Customers combined:", most_customers_country['Country'])

Country with the most VIP and Preferred Customers combined: United Kingdom
